In [1]:
import numpy as np
import pandas as pd
import string
import calendar
import re
import math
import sympy
import matplotlib.pyplot as plt
import pprint
import itertools

from fractions import Fraction
from scipy import stats
from scipy import special


print(f"numpy version is {np.__version__}")
print(f"pandas version is {pd.__version__}")


numpy version is 1.20.3
pandas version is 1.3.2


### failure rate gives probability of extra lifetime

In [35]:
# symbolic functions and integration
low, high   = 1, 2
t           = sympy.Symbol('t')
e_arg       = sympy.integrate(t ** 3, (t, low, high))
surv_prob   = math.pow(math.e, -e_arg)

print(f'Probability of age {low} surviving to age {high} is {surv_prob * 100} %')

Probability of age 1 surviving to age 2 is 2.3517745856009116 %


### stopping at $r^{th}$ failure

In [13]:
fail_times      = np.fromstring('.35, .73, .99, 1.40, 1.45, 1.83, 2.20, 2.72', sep=',')
n               = 20
r               = fail_times.shape[0]


stopTime        = np.max(fail_times)
timeOnTest      = (np.sum(fail_times) + (n-r) * stopTime)
theta_hat       = timeOnTest / r


# hypothesis testing
sig             = 5
theta_0         = 10
TS              = 2 * timeOnTest / theta_0
p_val           = 2 * min(stats.chi2.cdf(TS, 2*r), 1 - stats.chi2.cdf(TS, 2*r))
status          = 'accepted' if (100 * p_val >= sig) else 'rejected'

conf            = .95
conf_low        = 2 * timeOnTest / (stats.chi2.ppf(0.5 * (1 + conf), 2*r))
conf_high       = 2 * timeOnTest / (stats.chi2.ppf(1 - 0.5 * (1 + conf), 2*r))

64.61245783508774

#### print output tables

In [118]:
outDict     ={'Total items ($n$)'                   : '{:d}'.format(n),
            'Number failed ($r$)'                   : '{:d}'.format(r),
            'Total time on test ($\\tau$)'          : '{:.2f}'.format(timeOnTest),
            'MLE of mean ($\\hat{\\theta}$)'        : '{:.2f}'.format(theta_hat),
            'Stop time ($T$)'                       : '{:.2f}'.format(stopTime),
            '$p$ value \%'                          : '{:.2f}'.format(100 * p_val),
            }
dfb = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])

print(f'Confidence interval of {conf} \% for the mean lifetime is [{round(conf_low, 2)}, {round(conf_high, 2)}]')
print(f'Hypothesis test for $\\theta_0$ = {theta_0} is {status}\n')

s4          = dfb.to_latex(escape=False, column_format='@{}lr@{}')
s4          = s4.replace('{} &    Value', '\multicolumn{2}{c}{\\texttt{Stopping at \\emph{r} failures}}\n')
print('\\begin{table}[H]')
print('\\centering')
print(s4)
print('\\end{table}')
print('\\bigskip')


44.31

### mean and variance of testing period

In [10]:
n, r                    = 20, 10
j                       = np.arange(r) + 1
exponMean               = 10
testPeriod_mean_exact   = np.sum(np.divide(exponMean, n+1 - j))
testPeriod_var_exact    = np.sum(np.divide(exponMean, n+1 - j) ** 2)


outDict     ={'Total items ($n$)'                   : '{:d}'.format(n),
            'Number failed ($r$)'                   : '{:d}'.format(r),
            'Mean lifetime ($\\theta$)'             : '{:.2f}'.format(exponMean),
            'Test Period mean (exact) '             : '{:.2f}'.format(testPeriod_mean_exact),
            'Test Period variance (exact) '         : '{:.2f}'.format(testPeriod_var_exact),
            }
dfb = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])


s4          = dfb.to_latex(escape=False, column_format='@{}lr@{}')
s4          = s4.replace('{} &  Value', '\multicolumn{2}{c}{\\texttt{Statistics of testing period}}\n')
print('\\begin{table}[H]')
print('\\centering')
print(s4)
print('\\end{table}')
print('\\bigskip')

\begin{table}[H]
\centering
\begin{tabular}{@{}lr@{}}
\toprule
\multicolumn{2}{c}{\texttt{Statistics of testing period}}
 \\
\midrule
Total items ($n$)             &     20 \\
Number failed ($r$)           &     10 \\
Mean lifetime ($\theta$)      &  10.00 \\
Test Period mean (exact)      &   6.69 \\
Test Period variance (exact)  &   4.64 \\
\bottomrule
\end{tabular}

\end{table}
\bigskip


### sequential testing

In [14]:
# fail_times      = np.fromstring('.35, .73, .99, 1.40, 1.45, 1.83, 2.20, 2.72', sep=',')
# r               = fail_times.shape[0]

r               = 16
timeOnTest      = 300

theta_hat       = timeOnTest / r


# hypothesis testing
sig             = 5
theta_0         = 20
TS              = 2 * timeOnTest / theta_0
p_val           = 2 * min(stats.chi2.cdf(TS, 2*r), 1 - stats.chi2.cdf(TS, 2*r))
status          = 'accepted' if (100 * p_val >= sig) else 'rejected'

# confidence interval for theta
conf            = .95
conf_low        = 2 * timeOnTest / (stats.chi2.ppf(0.5 * (1 + conf), 2*r))
conf_high       = 2 * timeOnTest / (stats.chi2.ppf(1 - 0.5 * (1 + conf), 2*r))

#### print output tables

In [22]:
outDict     ={'Number failed ($r$)'                 : '{:d}'.format(r),
            'Total time on test ($T$)'              : '{:.2f}'.format(timeOnTest),
            'MLE of mean ($\\hat{\\theta}$)'        : '{:.2f}'.format(theta_hat),
            '$p$ value \%'                          : '{:.2f}'.format(100 * p_val)
            }
dfb = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])

print(f'Confidence interval of {100 * conf} \% for the mean lifetime is [{round(conf_low, 2)}, {round(conf_high, 2)}]\n')
print(f'Hypothesis test for $\\theta_0$ = {theta_0} is {status}\n')

s4          = dfb.to_latex(escape=False, column_format='@{}lr@{}')
s4          = s4.replace('{} &   Value', '\multicolumn{2}{c}{\\texttt{Stopping at fixed time}}\n')
print('\\begin{table}[H]')
print('\\centering')
print(s4)
print('\\end{table}')
print('\\bigskip')

Confidence interval of 95.0 \% for the mean lifetime is [12.13, 32.8]

Hypothesis test for $\theta_0$ = 20 is accepted

\begin{table}[H]
\centering
\begin{tabular}{@{}lr@{}}
\toprule
\multicolumn{2}{c}{\texttt{Stopping at fixed time}}
 \\
\midrule
Number failed ($r$)          &      16 \\
Total time on test ($T$)     &  300.00 \\
MLE of mean ($\hat{\theta}$) &   18.75 \\
$p$ value \%                 &   86.38 \\
\bottomrule
\end{tabular}

\end{table}
\bigskip


### simultaneous testing with fixed time stop

In [29]:
fail_times      = np.fromstring('15, 28.2, 46, 62.2, 76, 86, 128, 153, 197', sep=',')
lifetimes       = np.hstack((fail_times[0], np.diff(fail_times)))
n               = 10
r               = fail_times.shape[0]
stopTime        = 200
panelSize       = 5


if panelSize == n:
    timeOnTest      = (n-r) * stopTime + np.sum(fail_times)
elif panelSize < n:
    timeOnTest      = 0
    for j in range(panelSize + 1, lifetimes.shape[0]):
        timeOnTest += (r+1-j) * lifetimes[j]
    if r < n:
        timeOnTest += (stopTime - fail_times[-1]) * (n-r)
    timeOnTest += fail_times[panelSize] * panelSize
    
theta_hat       = timeOnTest / r


# hypothesis testing
sig             = 5
theta_0         = 10
TS              = 2 * timeOnTest / theta_0
p_val           = 2 * min(stats.chi2.cdf(TS, 2*r), 1 - stats.chi2.cdf(TS, 2*r))
status          = 'accepted' if (100 * p_val >= sig) else 'rejected'

conf            = .95
conf_low        = 2 * timeOnTest / (stats.chi2.ppf(0.5 * (1 + conf), 2*r))
conf_high       = 2 * timeOnTest / (stats.chi2.ppf(1 - 0.5 * (1 + conf), 2*r))

outDict     ={'Number failed ($r$)'                 : '{:d}'.format(r),
            'Total time on test ($T$)'              : '{:.2f}'.format(timeOnTest),
            'MLE of mean ($\\hat{\\theta}$)'        : '{:.2f}'.format(theta_hat),
            '$p$ value \%'                          : '{:.2f}'.format(100 * p_val)
            }
dfb = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])

print(f'Confidence interval of {100 * conf} \% for the mean lifetime is [{round(conf_low, 2)}, {round(conf_high, 2)}]\n')
print(f'Hypothesis test for $\\theta_0$ = {theta_0} is {status}\n')

s4          = dfb.to_latex(escape=False, column_format='@{}lr@{}')
s4          = s4.replace('{} &   Value', '\multicolumn{2}{c}{\\texttt{Stopping at fixed time}}\n')
print('\\begin{table}[H]')
print('\\centering')
print(s4)
print('\\end{table}')
print('\\bigskip')

Confidence interval of 95.0 \% for the mean lifetime is [48.47, 185.65]

Hypothesis test for $\theta_0$ = 10 is rejected

\begin{table}[H]
\centering
\begin{tabular}{@{}lr@{}}
\toprule
\multicolumn{2}{c}{\texttt{Stopping at fixed time}}
 \\
\midrule
Number failed ($r$)          &       9 \\
Total time on test ($T$)     &  764.00 \\
MLE of mean ($\hat{\theta}$) &   84.89 \\
$p$ value \%                 &    0.00 \\
\bottomrule
\end{tabular}

\end{table}
\bigskip


In [15]:
lifetimes

array([15. , 13.2, 17.8, 16.2, 13.8, 10. , 42. , 25. , 44. ])

In [37]:
ended = np.array([1.2, 2.2, 4.1, 5.6, 8.4, 16.2, 21.7, 41, 49.3, 60.5, 94, 98])
cont  = np.array([1.8, 11.8, 13.4, 29, 42, 42.4, 61, 99.2])

(np.sum(ended) + np.sum(cont)) / ended.shape[0]


58.56666666666666

### two sample problem

In [49]:
fa      = np.fromstring('212, 88.5, 122.3, 116.4, 125, 132, 66', sep=',')
fb      = np.fromstring('34.6, 54, 162, 49, 78, 121, 128', sep=',')

TS_F    = (np.mean(fa) / np.mean(fb)) * (fb.shape[0] / fa.shape[0])

sig     = 5
p_val_F   = 2 * min(stats.f.cdf(TS_F, fa.shape[0], fb.shape[0]), 1 - stats.f.cdf(TS_F, fa.shape[0], fb.shape[0]))


# output results as table
outDict     ={'Sample A size ($n$)'                 : '{:d}'.format(fa.shape[0]),
              'Sample B size ($m$)'                 : '{:d}'.format(fb.shape[0]),
            'Mean Lifetime A ($\\overline{X}$)'     : '{:.2f}'.format(np.mean(fa)),
            'Mean Lifetime B ($\\overline{Y}$)'     : '{:.2f}'.format(np.mean(fb)),
            'Test Statistic (F Test)'               : '{:.2f}'.format(TS_F),
            '$p$ vlaue \% '                         : '{:.2f}'.format(100 * p_val_F)
            }
dfb = pd.DataFrame.from_dict(outDict, orient='index', columns = ['Value'])


s4          = dfb.to_latex(escape=False, column_format='@{}lr@{}')
s4          = s4.replace('{} &   Value', '\multicolumn{2}{c}{\\texttt{Two sample problem}}\n')
print('\\begin{table}[H]')
print('\\centering')
print(s4)
print('\\end{table}')
print('\\bigskip')


\begin{table}[H]
\centering
\begin{tabular}{@{}lr@{}}
\toprule
\multicolumn{2}{c}{\texttt{Two sample problem}}
 \\
\midrule
Sample A size ($n$)              &       7 \\
Sample B size ($m$)              &       7 \\
Mean Lifetime A ($\overline{X}$) &  123.17 \\
Mean Lifetime B ($\overline{Y}$) &   89.51 \\
Test Statistic (F Test)          &    1.38 \\
$p$ vlaue \%                     &   68.43 \\
\bottomrule
\end{tabular}

\end{table}
\bigskip


In [40]:
702/7


100.28571428571429